In [1]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from scipy.signal import butter, filtfilt
from sklearn.metrics import roc_curve
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import rcParams

import h5py, numpy as np
import os

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [ ]:
fig_dir = "./figures"
data_dir = "./data"
results_dir = "./results"

os.makedirs(fig_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

### EDAD

In [3]:
class EDAD():

    def __init__(self, lr, mom_score, mode, n_learning, device='cpu'):
        self.lr = lr
        self.mom_score = mom_score
        self.mode = mode
        self.n_learning = n_learning
        self.device = device

    def fit(self, X, y=None):

        device = self.device

        X = torch.tensor(X, dtype=torch.float).to(device)
        R = torch.eye(X.shape[1], dtype=torch.float).to(device)
        self.p = (1.0) / (X.shape[1] - 1)

        # ---------------------------------------------------------
        #  MODE: UNSUPERVISED ('u')
        # ---------------------------------------------------------
        if self.mode == 'u':

            NR = torch.zeros(X.shape[0]).to(device)

            for i in tqdm(range(X.shape[0]), desc="EDAD (unsupervised)"):
                XD = F.linear(X[i:i+1, :], R)
                XD = (XD.T @ XD) / len(XD)
                XD -= torch.diag(torch.diag(XD))
                XD = (self.p * XD @ R)
                XD = self.lr * XD
                R -= XD
                NR[i] = torch.norm(R)

            NR = np.abs(np.gradient(NR.cpu()))
            scores = np.zeros(len(NR))
            scores[0] = NR[0]

            for i in range(len(NR)):
                scores[i] = (1 - self.mom_score) * scores[i - 1] + self.mom_score * NR[i]

            decision_scores = torch.nan_to_num(torch.tensor(scores), nan=0.0, posinf=0.0, neginf=0.0)

            self.decision_scores_ = decision_scores

        # ---------------------------------------------------------
        #  MODE: SEMI-SUPERVISED ('s')
        # ---------------------------------------------------------
        elif self.mode == 's':

            NR = torch.zeros(X.shape[0] - self.n_learning).to(device)

            for i in tqdm(range(X.shape[0]), desc="EDAD (semi-supervised)"):
                XD = F.linear(X[i:i+1, :], R)
                XD = (XD.T @ XD) / len(XD)
                XD -= torch.diag(torch.diag(XD))
                XD = (self.p * XD @ R)

                if i > self.n_learning - 1:
                    NR[i - self.n_learning] = torch.mean(XD)
                else:
                    R -= (self.lr * XD)
                    if i == self.n_learning - 1:
                        print("Stopping R updates at step", i)

            self.decision_scores_ = NR
        else:

            NR = torch.zeros(X.shape[0]).to(device)

            for i in tqdm(range(X.shape[0]), desc="EDAD (unsupervised)"):
                XD = F.linear(X[i:i+1, :], R)
                XD = (XD.T @ XD) / len(XD)
                XD -= torch.diag(torch.diag(XD))
                XD = (self.p * XD @ R)
                XD = self.lr * XD
                R -= XD

            self.decorr_energy = R


### Loading LHC dataset

In [4]:
def load_events(path):
    with h5py.File(path, "r") as f:
        X = f["Particles"][...]
    return X


X_bg  = load_events(os.path.join(data_dir, "background_for_training.h5"))
X_bg = X_bg[:, :, :-1].reshape(X_bg.shape[0], -1)

X_sig1 = load_events(os.path.join(data_dir, "leptoquark_LOWMASS_lepFilter_13TeV.h5"))
X_sig1 = X_sig1[:, :, :-1].reshape(X_sig1.shape[0], -1)

X_sig2 = load_events(os.path.join(data_dir, "Ato4l_lepFilter_13TeV.h5"))
X_sig2 = X_sig2[:, :, :-1].reshape(X_sig2.shape[0], -1)

X_sig3 = load_events(os.path.join(data_dir, "hToTauTau_13TeV_PU20.h5")) 
X_sig3 = X_sig3[:, :, :-1].reshape(X_sig3.shape[0], -1)

X_sig4 = load_events(os.path.join(data_dir, "hChToTauNu_13TeV_PU20.h5"))
X_sig4 = X_sig4[:, :, :-1].reshape(X_sig4.shape[0], -1)



print("Background shape: ", X_bg.shape)
print("Signal1 shape: ", X_sig1.shape)
print("Signal2 shape: ", X_sig2.shape)
print("Signal3 shape: ", X_sig3.shape)
print("Signal4 shape: ", X_sig4.shape)

Background shape:  (4000000, 57)
Signal1 shape:  (340544, 57)
Signal2 shape:  (55969, 57)
Signal3 shape:  (691283, 57)
Signal4 shape:  (760272, 57)


### Using a randomly selected training window of train_length = 80,000 samples (≈2% of the total 4M samples) as a small background slice for grid search.

In [5]:
train_length = 80000

# X_bg_train, X_bg_test = train_test_split(
#     X_bg, 
#     train_size=train_length, 
#     # test_size=80000, 
#     shuffle=True, 
#     random_state=None
# )

In [6]:
# save train and test splits
# np.save(os.path.join(data_dir,"LHC_L1T_X_bg_train.npy"), X_bg_train)
# np.save(os.path.join(data_dir,"LHC_L1T_X_bg_test.npy"), X_bg_test)

# load train and test splits
X_bg_train = np.load(os.path.join(data_dir,"LHC_L1T_X_bg_train.npy"))
X_bg_test = np.load(os.path.join(data_dir,"LHC_L1T_X_bg_test.npy"))

### Grid Search

In [7]:
learning_rate_list = [8e-6, 2e-6, 8e-7, 2e-7, 8e-8, 2e-8]

best_lr = None
min_diff = 1e9

for learning_rate in learning_rate_list:

    EDAD_model = EDAD(lr=learning_rate, mom_score=0.25, mode='e', n_learning=train_length, device="cpu")

    EDAD_model.fit(X_bg_train)
    Decorrelation_transform = EDAD_model.decorr_energy

    X_bg_train_tensor = torch.tensor(X_bg_train[len(X_bg_train)//2:], dtype=torch.float)
    corr_matrix_bg = (X_bg_train_tensor.T @ X_bg_train_tensor) / len(X_bg_train_tensor)
    corr_matrix_bg -= torch.diag(torch.diag(corr_matrix_bg))
    corr_matrix_bg = corr_matrix_bg @ Decorrelation_transform

    diff = torch.norm(corr_matrix_bg)

    if diff < min_diff:
        min_diff = diff
        best_lr = learning_rate
        print("New best learning rate found:", best_lr, "with diffence:", min_diff)

print("Best learning rate is:", best_lr, "with diffence:", min_diff)

EDAD (unsupervised):   0%|          | 0/80000 [00:00<?, ?it/s]

EDAD (unsupervised): 100%|██████████| 80000/80000 [00:08<00:00, 9412.47it/s]


New best learning rate found: 8e-06 with diffence: tensor(2368.2937)


EDAD (unsupervised): 100%|██████████| 80000/80000 [00:08<00:00, 9312.66it/s]


New best learning rate found: 2e-06 with diffence: tensor(2347.0867)


...

New best learning rate found: 8e-07 with diffence: tensor(2196.2815)


EDAD (unsupervised): 100%|██████████| 80000/80000 [00:08<00:00, 9410.12it/s]


New best learning rate found: 2e-07 with diffence: tensor(1966.4988)


EDAD (unsupervised): 100%|██████████| 80000/80000 [00:08<00:00, 9446.62it/s]

Best learning rate is: 2e-07 with diffence: tensor(1966.4988)
